# Compare W2V models results over eval.set
## TODO: Weighted mean of vectors

In [1]:
# load eval.set (duplicates free)
import json
filename = 'pre-clustered_docs_harder.json'
with open(filename, 'r') as file:
    cdocs = json.load(file)


# Load entities models
## TODO: lower case text

In [2]:
# load pre-trained model
from gensim.models import Word2Vec

unwrapped_docs = [doc for cluster in cdocs for doc in cluster]

#wiki_model = Word2Vec.load('./models/wiki_iter=5_algorithm=skipgram_window=10_size=300_neg-samples=10.m')
my_model = Word2Vec.load('./models/w2v_entities+abstract_model.model')
print("Models loaded")

Models loaded


# Evaluate Clustering

In [3]:
import utils
import my_dbscan
import model_evaluation as me
import w2v_model_utils as mm_utils
#import importlib
#importlib.reload(mm_utils)

# infer document vectors (once, they won't change)
my_doc_vecs = [mm_utils.infer_vector(doc['result_entities'], my_model) for doc in unwrapped_docs 
               if mm_utils.infer_vector(doc['result_entities'], my_model)!=[]]
#wiki_doc_vecs = [mm_utils.infer_vector(doc['result_entities'], wiki_model) for doc in unwrapped_docs]

titles = [doc['title'] for doc in unwrapped_docs]
urls = [doc['url'] for doc in unwrapped_docs]

models_recall = []
models_precision = []
clusters_found = [] # keep clusters results, they're useful later on
min_s = 2 # min_samples

# let's try 3 combinations of clustering
for k in range(3):
    # try different eps for some model
    if k == 0:
        # default eps dbscan
        eps = 0.27
        eps_increment = 0.1
    elif k==1:
        eps = 0.11
        eps_increment = 0.13
    else:
        # online update of model
        print("Online-update")
        eps = 0.11
        eps_increment = 0.13
        training_corpus = [utils.lower_case_list(doc['result_entities']) for doc in unwrapped_docs
                          if mm_utils.infer_vector(doc['result_entities'], my_model)!=[]]
        %time my_model = mm_utils.online_training(my_model, training_corpus)
        # wiki_model = mm_utils.online_training(wiki_model, training_corpus)
        # re-infer vectors
        my_doc_vecs = [mm_utils.infer_vector(doc['result_entities'], my_model) for doc in unwrapped_docs]
        #wiki_doc_vecs = [mm_utils.infer_vector(doc['result_entities'], wiki_model) for doc in unwrapped_docs]
    doc_vecs = my_doc_vecs
    # TODO: change to 2 to cluster both models
    for i in range(1):
        # apply dbscan clustering to these vectors
        urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                                   urls = urls, subset_length = len(titles),
                                                     eps = eps, eps_increment = eps_increment,
                                                   n_iterations = 3, verbose = False, min_samples = min_s)
        # get clusters as list of titles
        titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
        clusters_found.append(titles_clusters)
        #data = utils.plot_clusters(titles_clusters)

        # evaluate clustering
        expected_clusters = []
        for docs in cdocs:
            expected_clusters.append([doc['title'] for doc in docs])
        precision, recall = me.compute_clustering_accuracy(titles_clusters, expected_clusters)
        print('Precision score: %s, Recall score: %s'%(precision, recall))

        models_precision.append(precision)
        models_recall.append(recall)
        # last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
        # that's way we have precision score
        print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
            len(urls_cluster_list), len(cdocs)))
        # change inferred docs for next model
        #doc_vecs = wiki_doc_vecs

Results occurences(correct guess, cluster index):  [(4, 0), (3, 0), (3, 0), (2, 0), (10, 0), (7, 0), (3, 0), (4, 0), (10, 0), (6, 0)]
Accuracy (Precision) over each cluster:  [7.6923076923076925, 5.769230769230769, 5.769230769230769, 3.8461538461538463, 19.23076923076923, 13.461538461538462, 5.769230769230769, 7.6923076923076925, 19.23076923076923, 11.538461538461538]
Accuracy (Recall) over each cluster:  [100.0, 100.0, 100.0, 100.0, 100.0, 77.77777777777777, 100.0, 100.0, 100.0, 100.0]
Precision score: 9.999999999999998, Recall score: 97.77777777777779
#Number of clusters found: 1, against number of pre-computed clusters: 10#

Results occurences(correct guess, cluster index):  [(4, 0), (2, 1), (3, 1), (2, 0), (10, 1), (6, 2), (3, 1), (4, 0), (10, 1), (6, 1)]
Accuracy (Precision) over each cluster:  [40.0, 5.714285714285714, 8.571428571428571, 20.0, 28.571428571428573, 100.0, 8.571428571428571, 40.0, 28.571428571428573, 17.142857142857142]
Accuracy (Recall) over each cluster:  [100.0, 

/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:

elementwise != comparison failed; this will raise an error in the future.

/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning:

elementwise != comparison failed; this will raise an error in the future.



In [11]:
for cluster in clusters_found[3]:
    print(cluster)
#print(unwrapped_docs[29])

{'fonte_dati': ['trend_analisys'], 'id': 'https://www.webnews.it/speciale/fifa-19/', 'ta_id': [6, 7], 'title': 'FIFA 19: novità, uscita e prezzo', 'abstract': 'Presentato ufficialmente in occasione della conferenza stampa all’E3 2018, FIFA 19 è la nuova iterazione del gioco di calcio sviluppato da EA Sports che costituisce il 27esimo titolo della popolare e vendutissima serie. Arriva sul mercato con tante novità, sia a livello di gameplay che delle competizioni disponibili, per sfidare ancora una volta PES 2019, il rivale di Konami.\n Novità e gameplay Modalità Carriera e campionati Il Viaggio FUT, ovvero FIFA Ultimate Team Comparto grafico Il testimonial in copertina: Cristiano Ronaldo La demo giocabile/a&gt; Prezzo e uscita \nFIFA 19 è stato nuovamente migliorato rispetto al precedente titolo della serie introducendo peraltro tante novità sia a livello di funzioni che di gameplay, che vanno a incrementare sostanzialmente le possibilità di gioco e la gestione in campo.\nNuovo sistema 

# Visualize results graphically

In [17]:
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3', api_key='FdMB4O6qCfciGDOnLvdQ')
graph_name = 'model_eval_entities_results'

model_descr = [model.comment for model in models]
# plot test-accuracy results, plus silhoutte scores (times 100, since all the values are in between 0-1)
trace0 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_precision,
    name='Precision',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Bar(
    x = model_descr + ['loaded model'],
    y = [s*100 for s in s_scores] + [0],
    name='Silhoutte score',
    marker=dict(
        color='rgb(204,50,100)',
    )
    
)

trace2 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_recall,
    name='Recall',
    marker=dict(
        color='rgb(155, 244, 66)',
    )
    
)
# precision, recall, silhoutte
data = [trace0, trace2,  trace1]
layout = go.Layout(
    title = 'Clustering Accuracy Results',
    xaxis=dict(
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)',
            
        ),
        tickangle = -45
    ),
    yaxis=dict(
        title='Clustering accuracy (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
   
    barmode='group',
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename=graph_name)

# Silhoutte score
Silhouette coefficient combines ideas of both cohesion and separation, 
but for individual points, as well as clusters and 
clusterings; the Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.
The Silhoutte score is merely the average of each silhoutte coefficient, computed over each sample.
It's a measure of the goodness of clustering, by assuming the fact that a cluster X is defined good if both every sample inside it is close to each other, and far from any other relatively-near cluster Y.
This is not necessarily true for every shape of cluster.

    The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.

In [16]:
from sklearn.metrics import silhouette_score
s_scores = []
for model in models:
    # get clusters for each model
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = 0.27, eps_increment = 0.1, n_iterations = 3, verbose = False)
    
    # get cluster labels, mantaining original docs ordering
    labels = []
    for doc in unwrapped_docs:
        noise = True
        for i, url_list in enumerate(urls_cluster_list):
            if(doc['url'] in url_list):
                labels.append(i) # keep cluster id
                noise = False
        if noise:
            labels.append(-1)
    # make sure they have the same size
    assert len(labels) == len(doc_vecs)
    ss = silhouette_score(doc_vecs, labels , metric='cosine')
    s_scores.append(ss)
    print(ss)

0.2057951
0.23791973
0.24932371
0.24040304
0.13758796
0.0041218703
0.12820731
0.228402


# Conclusions: 
A really low silhoutte score helps us identify models that tend to have very few cluster, hence not really recognizing differences between docs. This is fundamental, since test rules used so far prevent us from recognizing these kinds of models.
Since silhoutte score is an average of silhoutte coefficients, smaller changes of values here may mean greater differences.